In [ ]:
import numpy as np

# Радиус Земли в километрах
R = 6371.0

def haversine(lat1, lon1, lat2, lon2):
    """
    Быстрое векторизованное вычисление расстояния между точками.
    Все аргументы — float или numpy.ndarray одинаковой длины.
    Возвращает расстояние в километрах.
    """

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

In [ ]:
!pip install h3

from h3.api import basic_str as h3

def latlng_to_h3(lat, lng, resolution=7):
    return h3.latlng_to_cell(lat, lng, resolution)

geo_train["h3_7"] = geo_train.apply(lambda r: latlng_to_h3(r.lat, r.lng, 7), axis=1)
geo_test["h3_7"] = geo_test.apply(lambda r: latlng_to_h3(r.lat, r.lng, 7), axis=1)
geo_train["h3_8"] = geo_train.apply(lambda r: latlng_to_h3(r.lat, r.lng, 8), axis=1)
geo_test["h3_8"] = geo_test.apply(lambda r: latlng_to_h3(r.lat, r.lng, 8), axis=1)
geo_train["h3_9"] = geo_train.apply(lambda r: latlng_to_h3(r.lat, r.lng, 9), axis=1)
geo_test["h3_9"] = geo_test.apply(lambda r: latlng_to_h3(r.lat, r.lng, 9), axis=1)

In [ ]:
def count_in_radius(lat_points, lon_points, 
                    lat_ref, lon_ref, 
                    radius_km):
    """
    Возвращает массив длины len(lat_ref):
    сколько объектов из lat_points/lon_points находятся в радиусе radius_km.
    """

    # матрица расстояний: shape (N_ref, N_points)
    d = haversine(
        lat_ref[:, None], lon_ref[:, None],
        lat_points[None, :], lon_points[None, :]
    )
    
    # True/False → 1/0 → сумма по точкам
    return np.sum(d <= radius_km, axis=1)


In [ ]:
def mean_in_radius(lat_points, lon_points, values,
                   lat_ref, lon_ref, radius_km):
    """
    Возвращает среднее по признаку values для всех объектов points,
    которые находятся в радиусе radius_km для каждой ref-точки.
    Если в радиусе нет точек → np.nan.
    """

    d = haversine(
        lat_ref[:, None], lon_ref[:, None],
        lat_points[None, :], lon_points[None, :]
    )
    
    mask = d <= radius_km  # shape (N_ref, N_points)

    # сумма и количество
    sums = np.sum(values[None, :] * mask, axis=1)
    counts = np.sum(mask, axis=1)

    # делим, где count>0, иначе nan
    mean_vals = np.where(counts > 0, sums / counts, np.nan)
    return mean_vals


In [ ]:
def nearest_k(lat_points, lon_points,
              lat_ref, lon_ref, k=5):
    """
    Возвращает:
    - d_k: расстояния до k ближайших (shape N_ref, k)
    - idx_k: индексы k ближайших (shape N_ref, k)
    """

    d = haversine(
        lat_ref[:, None], lon_ref[:, None],
        lat_points[None, :], lon_points[None, :]
    )

    idx = np.argpartition(d, kth=k-1, axis=1)[:, :k]

    # сортируем внутри k
    row_indices = np.arange(d.shape[0])[:, None]
    d_k = d[row_indices, idx]
    order = np.argsort(d_k, axis=1)

    idx_k = idx[row_indices, order]
    d_k = d_k[row_indices, order]

    return d_k, idx_k


In [ ]:
from sklearn.neighbors import BallTree

tree = BallTree(np.radians(np.c_[lat_points, lon_points]), metric='haversine')
dist, idx = tree.query(np.radians(np.c_[lat_ref, lon_ref]), k=1)
dist = dist[:,0] * 6371.0
idx = idx[:,0]


In [ ]:
import numpy as np

def polygon_area(lat, lon):
    """
    Площадь полигона в квадратных километрах.
    lat, lon — массивы координат вершин (в градусах).
    Используем формулу shoelace + поправку на кривизну поверхности.
    """

    # переводим в радианы
    lat = np.radians(lat)
    lon = np.radians(lon)

    # переводим lat/lon в "квазиплоские" координаты (проекция equirectangular)
    R = 6371  # радиус земли, км
    x = R * lon * np.cos(np.mean(lat))
    y = R * lat

    # shoelace formula
    area = 0.5 * np.abs(
        np.dot(x, np.roll(y, -1)) - np.dot(y, np.roll(x, -1))
    )
    return area


In [ ]:
def kmeans_geo(lat, lon, k=20, iters=20):
    """
    Простая и быстрая k-means кластеризация на координатах.
    Возвращает:
    cluster_id (N,)
    centroids_lat (k,)
    centroids_lon (k,)
    dist_to_own_centroid (N,)
    """
    N = len(lat)
    pts = np.stack([lat, lon], axis=1)

    # случайные центроиды
    idx = np.random.choice(N, k, replace=False)
    centroids = pts[idx].copy()

    for _ in range(iters):
        # расстояние до центроидов (haversine)
        d = haversine(pts[:,0][:,None], pts[:,1][:,None],
                      centroids[:,0][None,:], centroids[:,1][None,:])

        labels = np.argmin(d, axis=1)

        # обновление центроидов
        for j in range(k):
            group = pts[labels == j]
            if len(group) > 0:
                centroids[j] = np.mean(group, axis=0)

    # финальные дистанции
    d_final = haversine(pts[:,0], pts[:,1],
                        centroids[labels,0], centroids[labels,1])

    return labels, centroids[:,0], centroids[:,1], d_final


In [ ]:
def group_centroid(lat, lon, group_ids):
    """
    Для каждой группы считает центроид (средние lat/lon).
    group_ids — массив длины N, например: 0,1,1,5,5,5,...
    Возвращает:
    centroids_lat[group_id]
    centroids_lon[group_id]
    """
    groups = np.unique(group_ids)

    cent_lat = {}
    cent_lon = {}

    for g in groups:
        mask = group_ids == g
        cent_lat[g] = lat[mask].mean()
        cent_lon[g] = lon[mask].mean()

    return cent_lat, cent_lon


In [ ]:
def centroids_for_objects(lat, lon, group_ids):
    cent_lat, cent_lon = group_centroid(lat, lon, group_ids)

    out_lat = np.array([cent_lat[g] for g in group_ids])
    out_lon = np.array([cent_lon[g] for g in group_ids])

    # расстояние до центра группы
    dist = haversine(lat, lon, out_lat, out_lon)

    return out_lat, out_lon, dist


In [ ]:
def aggregate_in_radius(lat_points, lon_points, values,
                        lat_ref, lon_ref, R_km):
    """
    Возвращает словарь:
    {
        'min': ...
        'max': ...
        'mean': ...
        'median': ...
        'std': ...
        'sum': ...
    }
    """

    d = haversine(lat_ref[:,None], lon_ref[:,None],
                  lat_points[None,:], lon_points[None,:])

    mask = d <= R_km  # (N_ref, N_points)

    res = {}

    # избегаем деления на ноль
    def safe(arr, fill=np.nan):
        return np.where(np.any(mask, axis=1), arr, fill)

    res["min"]    = safe(np.min(np.where(mask, values, np.inf), axis=1))
    res["max"]    = safe(np.max(np.where(mask, values, -np.inf), axis=1))
    res["mean"]   = safe(np.sum(values[None,:] * mask, axis=1) / np.sum(mask, axis=1))
    res["sum"]    = safe(np.sum(values[None,:] * mask, axis=1))
    res["std"]    = safe(np.sqrt(np.sum(mask * (values[None,:] - res["mean"][:,None])**2, axis=1)
                                 / np.sum(mask, axis=1)))

    # median вручную:
    med = []
    for i in range(mask.shape[0]):
        v = values[mask[i]]
        med.append(np.median(v) if len(v) > 0 else np.nan)
    res["median"] = np.array(med)

    return res


In [ ]:
def density(count, R_km):
    return count / (np.pi * (R_km**2))

def ratio_densities(count_small, R_small, count_big, R_big):
    dens_small = density(count_small, R_small)
    dens_big   = density(count_big, R_big)

    return dens_small / dens_big


In [ ]:
# EX
count_300 = count_in_radius(lat, lon, lat, lon, 0.3)
count_1000 = count_in_radius(lat, lon, lat, lon, 1.0)

ratio = ratio_densities(count_300, 0.3, count_1000, 1.0)


In [ ]:
import numpy as np
import pandas as pd


# ============================================================
#  БАЗОВЫЕ ФУНКЦИИ
# ============================================================

R_EARTH = 6371.0  # км


def haversine(lat1, lon1, lat2, lon2):
    """
    Быстрый векторизованный рассчёт расстояния между точками.
    Принимает numpy массивы или pandas Series.
    """
    lat1 = np.radians(lat1.astype(float))
    lon1 = np.radians(lon1.astype(float))
    lat2 = np.radians(lat2.astype(float))
    lon2 = np.radians(lon2.astype(float))

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    return R_EARTH * c


# ============================================================
#  ПЛОЩАДЬ ПОЛИГОНА
# ============================================================

def polygon_area(lat_list, lon_list):
    """
    Вычисляет площадь полигона по координатам вершин.
    lat_list, lon_list — списки или массивы, длина одинакова.
    """

    lat = np.radians(np.array(lat_list))
    lon = np.radians(np.array(lon_list))

    R = R_EARTH
    x = R * lon * np.cos(lat.mean())
    y = R * lat

    area = 0.5 * np.abs(
        np.dot(x, np.roll(y, -1)) - np.dot(y, np.roll(x, -1))
    )
    return area


# ============================================================
#  RADIUS-BASED FEATURES
# ============================================================

def count_in_radius(df_points, df_ref, lat_col, lon_col, R_km):
    """
    Возвращает pd.Series длины df_ref:
    количество объектов df_points в радиусе R_km вокруг каждой точки df_ref.
    """
    lat_p = df_points[lat_col].to_numpy()
    lon_p = df_points[lon_col].to_numpy()
    lat_r = df_ref[lat_col].to_numpy()
    lon_r = df_ref[lon_col].to_numpy()

    d = haversine(lat_r[:, None], lon_r[:, None], lat_p[None, :], lon_p[None, :])
    return pd.Series(np.sum(d <= R_km, axis=1), index=df_ref.index)


def mean_in_radius(df_points, df_ref, lat_col, lon_col, value_col, R_km):
    """
    Среднее значения value_col среди объектов df_points в радиусе R_km.
    """
    lat_p = df_points[lat_col].to_numpy()
    lon_p = df_points[lon_col].to_numpy()
    vals = df_points[value_col].to_numpy()
    lat_r = df_ref[lat_col].to_numpy()
    lon_r = df_ref[lon_col].to_numpy()

    d = haversine(lat_r[:, None], lon_r[:, None], lat_p[None, :], lon_p[None, :])
    mask = d <= R_km

    sums = np.sum(vals[None, :] * mask, axis=1)
    counts = np.sum(mask, axis=1)

    out = np.where(counts > 0, sums / counts, np.nan)
    return pd.Series(out, index=df_ref.index)


def aggregate_in_radius(df_points, df_ref, lat_col, lon_col, value_col, R_km):
    """
    Полный набор агрегаций в радиусе:
    min / max / mean / median / std / sum
    Возвращает dict {name: pd.Series}
    """

    lat_p = df_points[lat_col].to_numpy()
    lon_p = df_points[lon_col].to_numpy()
    vals = df_points[value_col].to_numpy()
    lat_r = df_ref[lat_col].to_numpy()
    lon_r = df_ref[lon_col].to_numpy()

    d = haversine(lat_r[:, None], lon_r[:, None], lat_p[None, :], lon_p[None, :])
    mask = d <= R_km

    res = {}

    def safe(arr, fill=np.nan):
        return pd.Series(
            np.where(np.any(mask, axis=1), arr, fill),
            index=df_ref.index
        )

    res["min"] = safe(np.min(np.where(mask, vals, np.inf), axis=1))
    res["max"] = safe(np.max(np.where(mask, vals, -np.inf), axis=1))

    sums = np.sum(vals[None, :] * mask, axis=1)
    counts = np.sum(mask, axis=1)
    res["mean"] = safe(sums / counts)
    res["sum"] = safe(sums)

    std_arr = np.sqrt(
        np.sum(mask * (vals[None, :] - (sums / counts)[:, None]) ** 2, axis=1)
        / counts
    )
    res["std"] = safe(std_arr)

    medians = []
    for i in range(len(df_ref)):
        v = vals[mask[i]]
        medians.append(np.median(v) if len(v) > 0 else np.nan)
    res["median"] = pd.Series(medians, index=df_ref.index)

    return res


def density(count_series, R_km):
    return count_series / (np.pi * R_km ** 2)


def ratio_densities(count_small, R_small, count_big, R_big):
    dens_small = density(count_small, R_small)
    dens_big = density(count_big, R_big)
    return dens_small / dens_big


# ============================================================
#  БЛИЖАЙШИЕ ОБЪЕКТЫ
# ============================================================

def nearest_one(df_points, df_ref, lat_col, lon_col):
    """
    Возвращает две Series:
    - distance до ближайшего объекта
    - индекс ближайшего объекта в df_points
    """

    lat_p = df_points[lat_col].to_numpy()
    lon_p = df_points[lon_col].to_numpy()
    lat_r = df_ref[lat_col].to_numpy()
    lon_r = df_ref[lon_col].to_numpy()

    d = haversine(lat_r[:, None], lon_r[:, None], lat_p[None, :], lon_p[None, :])

    idx = np.argmin(d, axis=1)
    dist = np.min(d, axis=1)

    return (
        pd.Series(dist, index=df_ref.index),
        pd.Series(df_points.index[idx], index=df_ref.index)
    )


def nearest_with_value(df_points, df_ref, lat_col, lon_col, value_col):
    dist, idx = nearest_one(df_points, df_ref, lat_col, lon_col)
    nearest_vals = df_points[value_col].reindex(idx.values).values
    return dist, pd.Series(nearest_vals, index=df_ref.index)


# ============================================================
#  КЛАСТЕРИЗАЦИЯ
# ============================================================

def kmeans_geo(df, lat_col, lon_col, k=20, iters=20):
    """
    K-means на координатах DataFrame.
    Возвращает:
    - cluster_id Series
    - dist_to_centroid Series
    - centroids_lat, centroids_lon (массивы)
    """

    lat = df[lat_col].to_numpy()
    lon = df[lon_col].to_numpy()
    pts = np.stack([lat, lon], axis=1)
    N = len(df)

    # случайные центроиды
    idx = np.random.choice(N, k, replace=False)
    centroids = pts[idx].copy()

    for _ in range(iters):
        d = haversine(
            pts[:, 0][:, None], pts[:, 1][:, None],
            centroids[:, 0][None, :], centroids[:, 1][None, :]
        )
        labels = np.argmin(d, axis=1)

        for j in range(k):
            group = pts[labels == j]
            if len(group) > 0:
                centroids[j] = group.mean(axis=0)

    # финальные расстояния
    dist_to_centroid = haversine(
        pts[:, 0], pts[:, 1],
        centroids[labels, 0], centroids[labels, 1]
    )

    return (
        pd.Series(labels, index=df.index),
        pd.Series(dist_to_centroid, index=df.index),
        centroids[:, 0],
        centroids[:, 1]
    )


# ============================================================
#  ЦЕНТР МАСС ГРУПП
# ============================================================

def centroids_for_groups(df, lat_col, lon_col, group_col):
    """
    Возвращает:
    - centroid_lat Series
    - centroid_lon Series
    - dist_to_centroid Series
    """

    groups = df.groupby(group_col)

    cent_lat = groups[lat_col].transform("mean")
    cent_lon = groups[lon_col].transform("mean")

    dist = haversine(df[lat_col], df[lon_col], cent_lat, cent_lon)

    return cent_lat, cent_lon, pd.Series(dist, index=df.index)

In [ ]:
import geo_features as gf

# расстояние до ближайшего магазина
df["nearest_shop_dist"], df["nearest_shop_id"] = gf.nearest_one(shops, df, "lat", "lon")

# плотности в двух радиусах
df["count_300"] = gf.count_in_radius(df, df, "lat", "lon", 0.3)
df["count_1000"] = gf.count_in_radius(df, df, "lat", "lon", 1.0)

df["density_ratio"] = gf.ratio_densities(df["count_300"], 0.3,
                                         df["count_1000"], 1.0)

# кластеризация
df["cluster"], df["cluster_dist"], cent_lat, cent_lon = gf.kmeans_geo(df, "lat", "lon", k=15)

# центр масс компаний
df["company_lat"], df["company_lon"], df["company_dist"] = gf.centroids_for_groups(df, "lat", "lon", "company_id")